In [1]:
import pandas as pd
import numpy as np
import cplex

In [2]:
distance = pd.read_excel('Disttt.xlsx')
forecast_2018 = pd.read_excel('pred_18.xlsx')
forecast_2019 = pd.read_excel('pred_19.xlsx')

In [3]:
## reorganizing
distance = distance.drop(columns='Unnamed: 0')
forecast_2018 = forecast_2018.drop(columns='Unnamed: 0')
forecast_2019 = forecast_2019.drop(columns='Unnamed: 0')
forecast_2018.set_index('Index',drop=True,inplace=True)
forecast_2019.set_index('Index',drop=True,inplace=True)

In [4]:
#keys
farm = distance.index
farm_depot = distance.index
depot_refinary = distance.index
refinary = distance.index
distance.loc[5,7]

21.5222999999999

In [5]:
farm_depot_keys = [(f,d) for f in farm for d in farm_depot]
depot_refinary_keys = [(d,r) for d in depot_refinary for r in refinary ]

In [6]:
dist_dict = {(f,d): distance.loc[f,d] for f in farm for d in farm_depot}
dist_dict.get((5, 7))

21.5222999999999

In [7]:
# Create a CPLEX model
model = cplex.Cplex()

a = 0.001
b = 1
c = 1

In [8]:
# Define the variables
open_depot = { d : model.variables.add(obj=0, types=model.variables.type.binary, names=f'open_depot_{d}')  for d in farm_depot}
open_refinary = { d : model.variables.add(obj=0, types=model.variables.type.binary, names=f'open_refinary_{d}') for d in depot_refinary}
biomass_flows = {(f, d): model.variables.add(lb=0, types=model.variables.type.continuous, name=f'biomass_flows_{f}_{d}') for (f, d) in farm_depot_keys}
pellet_flows = {(d, r): model.variables.add(lb=0, types=model.variables.type.continuous, name=f'pellet_flows_{d}_{r}') for (d, r) in depot_refinary_keys}

TypeError: add() got an unexpected keyword argument 'name'

In [ ]:
# Set the objective function
Transport_cost_biomass = cplex.SparsePair(ind=[biomass_flows[(f, d)] for f in farm for d in farm_depot],
                                          val=[a * dist_dict[(f, d)] for f in farm for d in farm_depot])
Transport_cost_pellet = cplex.SparsePair(ind=[pellet_flows[(d, r)] for d in depot_refinary for r in refinary],
                                         val=[a * dist_dict[(d, r)] for d in depot_refinary for r in refinary])
Underutilization_cost_biomass = cplex.SparsePair(ind=[open_depot[d] for d in farm_depot],
                                                 val=[200.0 for _ in farm_depot])
Underutilization_cost_pellet = cplex.SparsePair(ind=[open_refinary[d] for d in depot_refinary],
                                                val=[700.0 for _ in depot_refinary])
cost_forecast = forecast_2018['mae'].sum()

In [ ]:
model.objective.set_sense(model.objective.sense.minimize)
model.objective.set_linear(list(Transport_cost_biomass.ind) + list(Transport_cost_pellet.ind) +
                           list(Underutilization_cost_biomass.ind) + list(Underutilization_cost_pellet.ind),
                           list(Transport_cost_biomass.val) + list(Transport_cost_pellet.val) +
                           list(Underutilization_cost_biomass.val) + list(Underutilization_cost_pellet.val))
model.objective.set_offset(cost_forecast)

In [ ]:
# Add constraints
for f in farm:
    for d in farm_depot:
        model.linear_constraints.add(lin_expr=[cplex.SparsePair(ind=[biomass_flows[(f,d)]], val=[1.0])],
                                     senses=["G"], rhs=[0.0], names=[f'bio_flow_{f}_{d}'])

for d in depot_refinary:
    for r in refinary:
        model.linear_constraints.add(lin_expr=[cplex.SparsePair(ind=[pellet_flows[(d, r)]], val=[1.0])],
                                     senses=["G"], rhs=[0.0], names=[f'pel_flow_{d}_{r}'])

In [ ]:
for d in farm_depot:
    model.linear_constraints.add(lin_expr=[cplex.SparsePair(ind=[biomass_flows[(f, d)] for f in farm], val=[1.0] * len(farm))],
                                 senses=["L"], rhs=[200.0], names=[f'depot_{d}_limit'])
for r in refinary:
    model.linear_constraints.add(lin_expr=[cplex.SparsePair(ind=[pellet_flows[(d, r)] for d in depot_refinary], val=[1.0] * len(depot_refinary))],
                                 senses=["L"], rhs=[700.0], names=[f'refinery_{r}_limit'])

In [ ]:
model.linear_constraints.add(lin_expr=[cplex.SparsePair(ind=[open_depot[d] for d in farm_depot], val=[1.0 for _ in farm_depot])],
                             senses=["G"], rhs=[15.0], names=['depot_limit_lower'])
model.linear_constraints.add(lin_expr=[cplex.SparsePair(ind=[open_refinary[d] for d in depot_refinary], val=[1.0 for _ in depot_refinary])],
                             senses=["G"], rhs=[2.0], names=['refinery_limit_lower'])

model.linear_constraints.add(lin_expr=[cplex.SparsePair(ind=[open_depot[d] for d in farm_depot], val=[1.0 for _ in farm_depot])],
                             senses=["L"], rhs=[20.0], names=['depot_limit_upper'])
model.linear_constraints.add(lin_expr=[cplex.SparsePair(ind=[open_refinary[d] for d in depot_refinary], val=[1.0 for _ in depot_refinary])],
                             senses=["L"], rhs=[5.0], names=['refinery_limit_upper'])

In [ ]:
for f in farm:
    model.linear_constraints.add(lin_expr=[cplex.SparsePair(ind=[open_depot[d] * biomass_flows[(f, d)] for d in farm_depot], val=[1.0 for _ in farm_depot])],
                                 senses=["G"], rhs=[0.8 * forecast_2018['forecast'].loc[f]], names=[f'farm_{f}_lower_limit'])

for f in farm:
    model.linear_constraints.add(lin_expr=[cplex.SparsePair(ind=[biomass_flows[(f, d)] for d in farm_depot], val=[1.0 for _ in farm_depot])],
                                 senses=["L"], rhs=[forecast_2018['forecast'].loc[f]], names=[f'farm_{f}_upper_limit'])

for f in farm:
    for d in farm_depot:
        model.linear_constraints.add(lin_expr=[cplex.SparsePair(ind=[biomass_flows[(f, d)], open_depot[d]], val=[-forecast_2018['forecast'].loc[f], forecast_2018['forecast'].loc[f]])],
                                     senses=["L", "G"], rhs=[0.0, 0.0], names=[f'link_{f}_{d}_bio'])


In [ ]:
for d in farm_depot:
    model.linear_constraints.add(lin_expr=[cplex.SparsePair(ind=[biomass_flows[(f, d)] for f in farm] + [pellet_flows[(d, r)] for r in refinary],
                                                             val=[1.0 for _ in farm] + [-open_refinary[d] for _ in refinary])],
                                 senses=["E"], rhs=[0.0], names=[f'balance_depot_{d}'])


In [ ]:
model.linear_constraints.add(lin_expr=[cplex.SparsePair(ind=[biomass_flows[(f, d)] for f in farm for d in farm_depot] + [pellet_flows[(d, r)] for d in depot_refinary for r in refinary],
                                                         val=[1.0 for _ in farm for _ in farm_depot] + [-1.0 for _ in depot_refinary for _ in refinary])],
                             senses=["E"], rhs=[0.0], names=['balance'])

In [ ]:
for d in farm_depot:
    model.linear_constraints.add(lin_expr=[cplex.SparsePair(ind=[open_depot[d], open_refinary[d]], val=[1.0, 1.0])],
                                 senses=["L"], rhs=[1.0], names=[f'one_depot_{d}'])

In [ ]:
# Set CPLEX parameters
model.parameters.mip.tolerances.mipgap.set(0.0)
model.parameters.timelimit.set(1800)
model.parameters.emphasis.mip.set(2)


In [ ]:
# Optimize the model
model.solve()


In [ ]:
# Get the optimization status
status = model.solution.get_status()
if status == cplex.solution.status.optimal:
    print("Optimal solution found.")
    # Get the objective value
    obj_value = model.solution.get_objective_value()
    print("Objective value:", obj_value)
    # Get the variable values
    open_depot_values = model.solution.get_values(open_depot.values())
    open_refinary_values = model.solution.get_values(open_refinary.values())
    biomass_flows_values = model.solution.get_values(biomass_flows.values())
    pellet_flows_values = model.solution.get_values(pellet_flows.values())
    # Print or use the variable values as needed
else:
    print("Optimization did not converge to an optimal solution.")